<a href="https://colab.research.google.com/github/anshupandey/AI_Agents/blob/main/AAP_C14_RAG_Evaluation_using_fixed_sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG Evaluation using Fixed Sources

A simple RAG pipeline requries at least two components: a retriever and a response generator. You can evaluate the whole chain end-to-end, as shown in the [QA Correctness](../qa-correctness/) walkthrough. However, for more actionable and fine-grained metrics, it is helpful to evaluate each component in isolation.

To evaluate the response generator directly, create a dataset with the user query and retrieved documents as inputs and the expected response as an output.

In this walkthrough, you will take this approach to evaluate the response generation component of a RAG pipeline, using both correctness and a custom "faithfulness" evaluator to generate multiple metrics. The results will look something like the following:

![Custom Evaluator](https://github.com/langchain-ai/langsmith-cookbook/blob/main/testing-examples/using-fixed-sources/img/example_results.png?raw=1)

## Prerequisites

First, install the required packages and configure your environment.

In [1]:
!pip install -q -U langchain-core langchain-community langgraph
!pip install --upgrade --quiet google-cloud-aiplatform requests
!pip install -q -U langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 2.6 MB/s eta 0:00:00

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

In [2]:
PROJECT_ID = "jrproject-402905"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [22]:
import os
import uuid

# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"  # Update with your API key
uid = uuid.uuid4()

## 1. Create a dataset

Next, create a dataset. The simple dataset below is enough to illustrate ways the response generator may deviate from the desired behavior by relying too much on its pretrained "knowledge".

In [4]:
# A simple example dataset
examples = [
    {
        "inputs": {
            "question": "What's the company's total revenue for q2 of 2022?",
            "documents": [
                {
                    "metadata": {},
                    "page_content": "In q1 the lemonade company made $4.95. In q2 revenue increased by a sizeable amount to just over $2T dollars.",
                }
            ],
        },
        "outputs": {
            "label": "2 trillion dollars",
        },
    },
    {
        "inputs": {
            "question": "Who is Lebron?",
            "documents": [
                {
                    "metadata": {},
                    "page_content": "On Thursday, February 16, Lebron James was nominated as President of the United States.",
                }
            ],
        },
        "outputs": {
            "label": "Lebron James is the President of the USA.",
        },
    },
]

In [5]:
from langsmith import Client

client = Client()

dataset_name = f"Faithfulness Example - {uid}"
dataset = client.create_dataset(dataset_name=dataset_name)
client.create_examples(
    inputs=[e["inputs"] for e in examples],
    outputs=[e["outputs"] for e in examples],
    dataset_id=dataset.id,
)

## 2. Define chain

Suppose your chain is composed of two main components: a retriever and response synthesizer. Using LangChain runnables, it's easy to separate these two components to evaluate them in isolation.

Below is a very simple RAG chain with a placeholder retriever. For our testing, we will evaluate ONLY the response synthesizer.

In [10]:
from langchain import prompts
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough

from langchain_google_vertexai import ChatVertexAI
llm = ChatVertexAI(model="chat-bison@002")

class MyRetriever(BaseRetriever):
    def _get_relevant_documents(self, query, *, run_manager):
        return [Document(page_content="Example")]


# This is what we will evaluate
response_synthesizer = prompts.ChatPromptTemplate.from_messages(
    [
        ("system", "Respond using the following documents as context:\n{documents}"),
        ("user", "{question}"),
    ]
) | llm

# Full chain below for illustration
chain = {
    "documents": MyRetriever(),
    "qusetion": RunnablePassthrough(),
} | response_synthesizer

## 3. Evaluate

Below, we will define a custom "FaithfulnessEvaluator" that measures how faithful the chain's output prediction is to the reference input documents, given the user's input question.

In this case, we will wrap the [Scoring Eval Chain](https://python.langchain.com/docs/guides/productionization/evaluation/string/scoring_eval_chain) and manually select which fields in the run and dataset example to use to represent the prediction, input question, and reference.

In [11]:
from langsmith.evaluation import RunEvaluator, EvaluationResult
from langchain.evaluation import load_evaluator


class FaithfulnessEvaluator(RunEvaluator):
    def __init__(self):
        self.evaluator = load_evaluator(
            "labeled_score_string",
            criteria={
                "faithful": "How faithful is the submission to the reference context?"
            },
            normalize_by=10,
        )

    def evaluate_run(self, run, example) -> EvaluationResult:
        res = self.evaluator.evaluate_strings(
            prediction=next(iter(run.outputs.values())),
            input=run.inputs["question"],
            # We are treating the documents as the reference context in this case.
            reference=example.inputs["documents"],
        )
        return EvaluationResult(key="labeled_criteria:faithful", **res)

In [23]:
from langchain.smith import RunEvalConfig
from langchain_core.prompts.prompt import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator

_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
Respond with CORRECT or INCORRECT:
Grade:
"""

PROMPT = PromptTemplate(
    input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE
)


evaluation_llm = ChatVertexAI(model="gemini-1.5-flash-001")

#qa_evaluator = LangChainStringEvaluator("qa", config={"llm": evaluation_llm, "prompt": PROMPT})
#context_qa_evaluator = LangChainStringEvaluator("context_qa", config={"llm": eval_llm})
#cot_qa_evaluator = LangChainStringEvaluator("cot_qa", config={"llm": eval_llm})

eval_config = RunEvalConfig(
    evaluators=['qa'],
    eval_llm = evaluation_llm,
    input_key="question",
)
results = client.run_on_dataset(
    llm_or_chain_factory=response_synthesizer,
    dataset_name=dataset_name,
    evaluation=eval_config,
)


View the evaluation results for project 'cooked-furniture-23' at:
https://smith.langchain.com/o/d902fd7a-b325-505a-adcb-ea98ad22246a/datasets/051c540f-83ca-457a-9b13-5a24c888a8f1/compare?selectedSessions=a6268638-c58b-48fe-a76a-e1a27c4d7f2b

View all tests for Dataset Faithfulness Example - 73fa72f9-f2a9-48b2-98b3-d1b16aa698f8 at:
https://smith.langchain.com/o/d902fd7a-b325-505a-adcb-ea98ad22246a/datasets/051c540f-83ca-457a-9b13-5a24c888a8f1
[>                                                 ] 0/2

[------------------------------------------------->] 2/2

## Discussion

You've now evaluated the response generator for its response correctness and its "faithfulness" to the source text but fixing retrieved document sources in the dataset. This is an effective way to confirm that the response component of your chat bot behaves according to expectations.

In setting up the evaluation, you used a custom run evaluator to select which fields in the dataset to use in the evaluation template. Under the hood, this still uses an off-the-shelf [scoring evaluator](https://python.langchain.com/docs/guides/productionization/evaluation/string/scoring_eval_chain).

Most of LangChain's open-source evaluators implement the "[StringEvaluator](https://python.langchain.com/docs/guides/productionization/evaluation/string/)" interface, meaning they compute a metric based on:

- An input string from the dataset example inputs (configurable by the RunEvalConfig's input_key property)
- An output prediction string from the evaluated chain's outputs (configurable by the RunEvalConfig's prediction_key property)
- (If labels or context are required) a reference string from the example outputs (configurable by the RunEvalConfig's reference_key property)

In our case, we wanted to take the context from the example _inputs_ fields. Wrapping the evaluator as a custom `RunEvaluator` is an easy way to get a further level of control in situations when you want to use other fields.